In [ ]:
import base64
import os
import requests
from dotenv import load_dotenv
from io import BytesIO
from openai import OpenAI
from pydantic import BaseModel

load_dotenv()
client = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### General

In [ ]:
response = client.chat.completions.create(
    model="gemma-3-27b-it",
    messages=[
        {
            # {"role": "system", "content": "You are a helpful assistant."},
            "role": "user",
            "content": "Explain how AI works in a few words"
        }
    ]
)
print(response.choices[0].message.content)

### Thinking

In [ ]:
response = client.chat.completions.create(
    model="gemini-2.5-flash",
    reasoning_effort="none",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Explain how AI works in a few words"
        }
    ]
)
print(response.choices[0].message.content)

### Streaming

In [ ]:
response = client.chat.completions.create(
  model="gemma-3-27b-it",
  messages=[
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ],
  stream=True
)

for chunk in response:
    print(chunk.choices[0].delta)

### Function Calling

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Chicago, IL",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            "required": ["location"],
            },
        }
    }
]

response = client.chat.completions.create(
  model="gemini-2.5-flash",
  messages=[{"role": "user", "content": "What's the weather like in Chicago, IL today?"}],
  tools=tools,
  tool_choice="auto"
)
print(response)

### Multimodal

In [ ]:
image_path = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"
response = requests.get(image_path)
image_bytes = BytesIO(response.content)
base64_image = base64.b64encode(image_bytes.getvalue()).decode()

response = client.chat.completions.create(
  model="gemma-3-27b-it",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)
print(response.choices[0].message.content)

### Structured Output

In [ ]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gemini-2.5-flash",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "John and Susan are going to an AI conference on Friday."},
    ],
    response_format=CalendarEvent
)

print(completion.choices[0].message.parsed)

### Embeddings

In [ ]:
response = client.embeddings.create(
    model="text-embedding-004",
    input="Your text string goes here"
)
print(response.data[0].embedding)